In [ ]:
# Read Bio-Rad ddPCR csv files and draw 2-dimensional amplitude plot. 
# Some targets are present at low abundance and yield very few positive droplets,
# making unsupervised GMM clustering unreliable. Because synthetic fluorescence-coded
# DNA templates generate the same droplet cluster locations as their corresponding
# mRNA targets (as demonstrated in Figure S2), we use ddPCR data from these synthetic
# templates to pre-define mRNA cluster locations. Similarly, protein cluster locations
# are defined using ddPCR data from standard curve measurements, although protein
# targets typically generate sufficient positive droplets in most of our tests.


import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
%matplotlib qt
# %matplotlib inline 

df1 = pd.read_csv('11plex_1.csv', usecols=["Ch1 Amplitude", "Ch2 Amplitude"]) 
df2 = pd.read_csv('11plex_2.csv', usecols=["Ch1 Amplitude", "Ch2 Amplitude"]) # df1 and df2 are datasets from two ddPCR reactions generated by splitting one sample into two aliquots to avoid high positive droplet occupancy
df3 = pd.read_csv('8plex_Synthetic DNA.csv', usecols=["Ch1 Amplitude", "Ch2 Amplitude"]) # df3 is used to define the cluster locations for each mRNA target
df4 = pd.read_csv('3plex_Protein.csv', usecols=["Ch1 Amplitude", "Ch2 Amplitude"]) # df4 is used to define the cluster locations for each protein target
df = pd.concat([df1, df2, df3, df4])

# swap two df columns
columns_titles = ["Ch2 Amplitude","Ch1 Amplitude"]
df=df.reindex(columns=columns_titles)

plt.figure(dpi=100)
y = df['Ch1 Amplitude']
x = df['Ch2 Amplitude']
plt.scatter(x,y,s=2,c='blue')
plt.xlabel('Ch2 Amplitude (HEX)', fontname='Arial', fontsize=15)  
plt.ylabel('Ch1 Amplitude (FAM)', fontname='Arial', fontsize=15)  
plt.xlim(0, 6500, 2000)
plt.ylim(0, 14000, 3000)

# Add grid
plt.grid(True, which='both', axis='both', linestyle="--")
plt.xticks(np.arange(0, 6501, 2000))
plt.yticks(np.arange(0, 14001, 3000))

# Change the font of the numbers on x and y axes
plt.xticks(fontname='Arial', fontsize=15)
plt.yticks(fontname='Arial', fontsize=15)

# Set background color
plt.gca().set_facecolor('white') 

In [ ]:
# Apply GMM with n_components=2 to separate negative droplets.
# Because negative droplets vastly outnumber positive droplets,
# the dominant Gaussian captures the negative population, allowing
# us to filter it out and prevent negative droplets from biasing
# downstream clustering of positive droplets.


from sklearn.mixture import GaussianMixture

# Fit GMM
gmm_neg = GaussianMixture(n_components=2, max_iter=1000, random_state=42, n_init=10, covariance_type='full').fit(df)
labels = gmm_neg.predict(df)
df['Clusters']=labels

# Plot
colors = np.array(["gray", "blue"])
for i in range(gmm_neg.n_components):
    cluster = df[df['Clusters']==i]
    cluster = cluster[['Ch2 Amplitude', 'Ch1 Amplitude']].values
    y0 = cluster[:,1]
    x0 = cluster[:,0]
    plt.scatter(x0,y0,s=2,c=colors[i])

plt.xlabel('Ch2 Amplitude (HEX)', fontname='Arial', fontsize=15)  
plt.ylabel('Ch1 Amplitude (FAM)', fontname='Arial', fontsize=15)  
plt.xlim(0, 6500, 2000)
plt.ylim(0, 14000, 3000)
plt.grid(True, which="both", axis="both", linestyle="--")
plt.xticks(np.arange(0, 6501, 2000), fontname="Arial", fontsize=15)
plt.yticks(np.arange(0, 14001, 3000), fontname="Arial", fontsize=15)
plt.gca().set_facecolor("white")
plt.show()


In [ ]:
# Apply GMM to identify all droplet clusters, including outliers.
# To improve GMM performance in resolving multiple clusters with uneven sizes,
# we explicitly specify the number of Gaussian components to be 12(11 + 1, where 
# 11 correspond to mRNA and protein targets and 1 represents the outlier population)
# and manually initialize the mean positions (x, y) for each component.
# Fine-tuning the initial means is very useful for accurate clustering.
# In this GMM model, the outliers can typically be identified by initializing 
# a Gaussian at the center of the "rain" droplet population.


df_pos_only = df[df['Clusters']==1][['Ch2 Amplitude', 'Ch1 Amplitude']] # df with negative droplets removed

# Fit GMM
means_init= np.array([[800, 6000], [750, 9700], [750, 11800],[3000, 1000], [4800, 1000], [3000, 6000], \
                    [4200, 6000], [5500, 6000], [2750, 11800], [3000, 9700], [4200, 9700], [3500, 7000]])
gmm_pos = GaussianMixture(n_components=12, max_iter=1000, random_state=42, n_init=10, means_init=means_init, covariance_type='full').fit(df_pos_only)
labels = gmm_pos.predict(df_pos_only)
df_pos_only['Clusters']=labels

# Plot
colors = np.array(["yellowgreen", "seagreen", "lightseagreen", "lightskyblue", "royalblue", "slateblue",\
                    "indigo", "navy", "tomato", "darkorange", "plum", "tan"])

for i in range(gmm_pos.n_components):
    cluster = df_pos_only[df_pos_only['Clusters']==i]
    cluster = cluster[['Ch2 Amplitude', 'Ch1 Amplitude']].values
    y0 = cluster[:,1]
    x0 = cluster[:,0]
    plt.scatter(x0,y0,s=2,c=colors[i])

plt.xlabel('Ch2 Amplitude (HEX)', fontname='Arial', fontsize=15)  
plt.ylabel('Ch1 Amplitude (FAM)', fontname='Arial', fontsize=15)  
plt.xlim(0, 6500, 2000)
plt.ylim(0, 14000, 3000)
plt.grid(True, which="both", axis="both", linestyle="--")
plt.xticks(np.arange(0, 6501, 2000), fontname="Arial", fontsize=15)
plt.yticks(np.arange(0, 14001, 3000), fontname="Arial", fontsize=15)
plt.gca().set_facecolor("white")
plt.show()


In [ ]:
# Extract positive droplets for each target cluster from the sample data (df_sample = df1 + df2).
# This is achieved by identifying datapoints shared between df_sample and df_pos_no_outliers
# (i.e., df_pos_only that excludes outliers), and propagating the cluster labels (and corresponding
# color codes) originally defined in df_pos_only to the matched sample droplets.


# Get the sample df (df1+df2)
df_sample = pd.concat([df1, df2], ignore_index=True).copy()
df_sample = df_sample.reindex(columns=["Ch2 Amplitude", "Ch1 Amplitude"])

df_pos_no_outliers = df_pos_only[df_pos_only["Clusters"] != 11].copy()
df_pos_no_outliers = df_pos_no_outliers.reindex(columns=["Ch2 Amplitude", "Ch1 Amplitude", "Clusters"])

# Exact overlapped droplets between df_sample and df_pos_no_outliers
df_exact_overlap = df_sample.merge(
    df_pos_no_outliers,
    on=["Ch2 Amplitude", "Ch1 Amplitude"],
    how="inner")

# Plot overlapped droplets by cluster
colors = np.array(["yellowgreen", "seagreen", "lightseagreen", "lightskyblue", "royalblue",
    "slateblue", "indigo", "navy", "tomato", "darkorange", "plum"])

plt.figure(dpi=100)
for i in range(11):
    sub = df_exact_overlap[df_exact_overlap["Clusters"] == i]
    plt.scatter(sub["Ch2 Amplitude"], sub["Ch1 Amplitude"], s=2, c=colors[i])

plt.xlabel('Ch2 Amplitude (HEX)', fontname='Arial', fontsize=15)  
plt.ylabel('Ch1 Amplitude (FAM)', fontname='Arial', fontsize=15)  
plt.xlim(0, 6500, 2000)
plt.ylim(0, 14000, 3000)
plt.grid(True, which="both", axis="both", linestyle="--")
plt.xticks(np.arange(0, 6501, 2000), fontname="Arial", fontsize=15)
plt.yticks(np.arange(0, 14001, 3000), fontname="Arial", fontsize=15)
plt.gca().set_facecolor("white")
plt.show()


In [ ]:
# Calculate target concentration for each cluster.
# For each target, the number of positive droplets is first counted,
# and the concentration is then computed using the following equation:
# C = −ln(1 − N_pos / N_total) / V_drop


# Droplet volume in uL (Bio-Rad ddPCR uses a droplet volume of 0.85 nL)
V_drop = 0.85e-3 

# Total droplets in sample
total_drops = len(df_sample)

# Count positive droplets per target cluster 
pos_counts = (
    df_exact_overlap["Clusters"]
    .value_counts()
    .sort_index()
)

# Compute concentration (copies/uL) using Poisson statistics
pos_fraction = pos_counts / total_drops
concentration_copies_per_uL = -np.log(1 - pos_fraction) / V_drop

# Assemble results table
results = pd.DataFrame({
    "Cluster": pos_counts.index,
    "pos_drops": pos_counts.values,
    "total_drops": total_drops,
    "pos_fraction": pos_fraction.values,
    "copies_per_uL": concentration_copies_per_uL.values
})

cluster_to_target = {
    0: "SMARCD3",
    1: "ICAM1",
    2: "EBI3",
    3: "HESX1",
    4: "FCER1A",
    5: "SUCLG2",
    6: "IFI27",
    7: "JUP",
    8: "CRP",
    9: "IP-10",
    10: "TRAIL"
}

results["Target"] = results["Cluster"].map(cluster_to_target)
print(results)
